# Extensions to the Basic Solow Model

Imports and set magics:

In [1]:


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
import sympy as sm
from types import SimpleNamespace
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from scipy.optimize import Bounds, minimize

# local modules
import modelproject as py

C:\Users\Peter\AppData\Local\Temp/ipykernel_2552/930102513.py:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')



## The Solow Model in Theory

We have the variables:

1. $K_t$ which is capital.
2. $L_t$ is labor with a constant growth rate of n.
3. $A_t$ is the labour augmenting technological progress, with a constant growth rate of g.
4. $Y_t = K_t^\alpha (A_tL_t)^{1-\alpha}$ is GDP, given a Cobb-Douglas production function.

Saving is a constant fraction of GDP, such that:
$$S_t = sY_t$$
Given this and assuming a constant rate of depreciation, capital accumulates according to:

$$K_{t+1} = sY_t + (1-\delta)K_t$$

The parameters of the model fulfill:
$$0<\alpha<1$$
$$0<s<1, 0<\delta<1$$
$$n\geq0, g\geq0$$

We define output and capital per effective worker as $y_t\equiv Y_t/(A_tL_t), k_t\equiv K_t/(A_tL_t)$ respectively.

The Solow model will be analyzed in terms of the variables expressed per effective worker, and the production function has constant return to scales.

The production function per effective worker is thus given by:
$$\frac{Y_t}{A_tL_t}=\frac{K_t^\alpha L_t^{1-\alpha}}{A_tL_t} \Leftrightarrow \tilde y_t = \tilde k_t^\alpha$$ 


From the capital accumulation equation, we divide both side by $A_{t+1}L_{t+1}$, and using that $A_{t+1}L_{t+1}=(1+n)(1+g)L_tA_t$ we get the transition equation for capital per effective worker:
$$\tilde k_{t+1} = \frac{1}{(1+n)(1+g)} [s \tilde y_t + (1-\delta) \tilde k_t]$$

In order to analyze the model, i will start by finding analytical expressions for the steady state values, $\tilde k^*,  \tilde y^*$, using that $\tilde k_{t+1} = \tilde k_t = \tilde k$ in steady state.

## Analytical solution

Finding an analytical expression for the steady states using sympy:
  

In [2]:
#Sympy symbols
k, y, alpha, delta, s, g, n = sm.symbols('k, y, alpha, delta, s, g, n')

#output per effective worker
y = k**alpha

#steady state:
ss = sm.Eq(k, 1/((1+n)*(1+g))*(s*y+(1-delta)*k))

#k steady state:
kss =sm.solve(ss,k)[0]
print('The steady state  of capital is given by:')
display(sm.Eq(sm.symbols('k^*'),kss))

#y steady state:
yss=kss**alpha
print('The steady state  of output is given by:')
display(sm.Eq(sm.symbols('y^*'),yss))

#creating function of steady state
ss_func = sm.lambdify((s,g,n,delta,alpha),kss)

# Evaluate function
print('The steady state value of capital using plausible parameter values is:')
display(ss_func(0.2,0.02,0.01,0.1,1/3))
print('The steady state value of output using plausible parameter values is:')
print(ss_func(0.2,0.02,0.01,0.1,1/3)**(1/3))

The steady state  of capital is given by:


Eq(k^*, ((delta + g*n + g + n)/s)**(1/(alpha - 1)))

The steady state  of output is given by:


Eq(y^*, (((delta + g*n + g + n)/s)**(1/(alpha - 1)))**alpha)

The steady state value of capital using plausible parameter values is:


1.903831539231319

The steady state value of output using plausible parameter values is:
1.2393943320395888


## Numerical solution

In this section we use the class SolowModelClass and then use the interactive function to vizualie the transition diagram of effective capital per worker. The initial parameter values are the same as in the analytical solution, and we thus expect the result to be the same.

In [3]:
#Plotting the transition diagram for the Solow model
print('kmin and kmax determines the starting points for transition of capital from below and above steady state respectively.\nSome of the parameter combinations may lead to high steady state values, and here kmax can be increased in order to show the full transition diagram')
#Interactive plot:
widgets.interact(py.interactive,
                
                kmin=widgets.FloatSlider(
                    description="$kmin$", min=0.0001, max=0.5, step=0.1, value=0.001),
                kmax=widgets.FloatSlider(
                    description="$kmax$", min=3, max=10000, step=1000, value=4),
                s=widgets.FloatSlider(
                    description="$s$", min=0.1, max=0.5, step=0.1, value=0.2),
                g=widgets.FloatSlider(
                    description="$g$", min=0.01, max=0.06, step=0.01, value=0.02),
                n=widgets.FloatSlider(
                    description="$n$", min=0.01, max=0.06, step=0.01, value=0.01),
                alpha=widgets.FloatSlider(
                     description="$alpha$", min=1/6, max=5/6, step=1/6, value=2/6),
                delta=widgets.FloatSlider(
                    description="$delta$", min=0.05, max=0.4, step=0.05, value=0.1),
               
)


kmin and kmax determines the starting points for transition of capital from below and above steady state respectively.
Some of the parameter combinations may lead to high steady state values, and here kmax can be increased in order to show the full transition diagram


interactive(children=(FloatSlider(value=0.001, description='$kmin$', max=0.5, min=0.0001), FloatSlider(value=4…

<function modelproject.interactive(kmin, kmax, s, g, n, alpha, delta)>

We see that capital will converge towards the steady state value. We also note that the steady state value is very sensitive to changing the parameter values. Below we plot the production function, which is in essence is output as a function of effective capital per worker.

In [4]:
#Interactive plot:
widgets.interact(py.interactive_output,
                s=widgets.FloatSlider(
                    description="$s$", min=0.1, max=0.5, step=0.1, value=0.2),
                g=widgets.FloatSlider(
                    description="$g$", min=0.01, max=0.06, step=0.01, value=0.02),
                n=widgets.FloatSlider(
                    description="$n$", min=0.01, max=0.06, step=0.01, value=0.01),
                alpha=widgets.FloatSlider(
                     description="$alpha$", min=1/6, max=5/6, step=1/6, value=2/6),
                delta=widgets.FloatSlider(
                    description="$delta$", min=0.05, max=0.3, step=0.05, value=0.1),
               
)

interactive(children=(FloatSlider(value=0.2, description='$s$', max=0.5, min=0.1), FloatSlider(value=0.02, des…

<function modelproject.interactive_output(s, g, n, alpha, delta)>

The production function will look messy depending on the parameter values. It looks rather realistic given the initial inputs, which are plausible.

## Extension of the general Solow model with human capital

In the following we will add human capital to the solow model. 

We thus have that 




We have the variables:

1. $K_t$ which is capital.
5. $H_t$ which is human capital.
2. $L_t$ is labor with a constant growth rate of n.
3. $A_t$ is the labour augmenting technological progress, with a constant growth rate of g.
4. $Y_t = K_t^\alpha H_t^\phi (A_tL_t)^{1-\alpha-\phi}$ is GDP, given a Cobb-Douglas production function.


Saving is a constant fraction of GDP, but we have that a fraction $s_h$ is spent on investment in human capital, and a fraction $s_k$ is spent on investment in physical capital.
$$S_t = (s)Y_t=(s_h + s_k)Y_t$$

The parameters of the model fulfill:
$$0<\alpha<1, 0 < \phi < 1$$
$$0<(s_h + s_k)<1,$$ 
$$0<\delta<1$$
$$n\geq0, g\geq0$$


We define output, capital and human capital per effective worker as $y_t\equiv Y_t/(A_tL_t), k_t\equiv K_t/(A_tL_t), h_t=H_t/(A_tL_t)=$ respectively.

The Solow model will be analyzed in terms of the variables expressed per effective worker, and the production function has constant return to scales.

The production function per effective worker is thus given by:
$$\frac{Y_t}{A_tL_t}=\frac{K_t^\alpha H_t^\phi L_t^{1-\alpha-\phi}}{A_tL_t} \Leftrightarrow y_t =  k_t^\alpha h_t^\phi$$ 


Like before, we can write the transition equation for capital and human capital per effective wroker as:
$$k_{t+1} = \frac{1}{(1+n)(1+g)} [s_k y_t + (1-\delta) k_t]$$
$$h_{t+1} = \frac{1}{(1+n)(1+g)} [s_h y_t + (1-\delta) h_t]$$

From these we can find the steady state values:

$$k^* = (\frac{s_k^{1-\phi}s_h^{\phi}}{n + g + \delta + ng})^\frac{1}{1-\alpha-\phi}$$
$$h^* = (\frac{s_k^{\alpha}s_h^{1-\alpha}}{n + g + \delta + ng})^\frac{1}{1-\alpha-\phi}$$

We tried to find steady state values of capital and human capital using scipy, but were unable to do so. Below we will graph the convergence of capital and human capital to the steady values given an initial value, and different model parameters.

In [5]:
#Interactive plot:
widgets.interact(py.interactive_solow_HC,
            
                s_h=widgets.FloatSlider(
                    description="$s_h$", min=0.1, max=0.4, step=0.1, value=0.2),
                s_k=widgets.FloatSlider(
                    description="$s_k$", min=0.1, max=0.4, step=0.1, value=0.1),
                g=widgets.FloatSlider(
                    description="$g$", min=0.01, max=0.06, step=0.01, value=0.02),
                n=widgets.FloatSlider(
                    description="$n$", min=0.01, max=0.06, step=0.01, value=0.01),
                alpha=widgets.FloatSlider(
                     description="$alpha$", min=1/7, max=3/7, step=1/7, value=2/6),
                delta=widgets.FloatSlider(
                    description="$delta$", min=0.05, max=0.3, step=0.05, value=0.1),
                phi=widgets.FloatSlider(
                    description="$phi$", min=1/7, max=3/7, step=1/7, value=2/6)

)

interactive(children=(FloatSlider(value=0.2, description='$s_h$', max=0.4, min=0.1), FloatSlider(value=0.1, de…

<function modelproject.interactive_solow_HC(s_h, s_k, g, n, alpha, delta, phi)>

# Conclusion

We see that the time of convergence of both capital per effective worker and human capital per effective wroker to their steady state values is dependent on the parameter values. 